In [11]:
import cv2
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
import os
import joblib
from sklearn.utils import shuffle
from sklearn.metrics import precision_score, recall_score, f1_score

In [12]:
def count_samples(labels):
    # Compter le nombre d'échantillons dans chaque classe
    unique_classes, class_counts = np.unique(labels, return_counts=True)

    # Afficher les résultats
    for emotion_label, count in zip(unique_classes, class_counts):
        print(f"Classe {emotion_label}: {count} échantillons")
##################################################################################

In [16]:
def process_data(data_folder, emotion_labels, images_list, labels_list, type):
    # Nombre maximum d'images à utiliser par classe émotionnelle
    max_images_per_class = 6000  # à retirer plus tard

    # Parcours des dossiers d'émotions
    for emotion_label in emotion_labels:
        emotion_folder = os.path.join(data_folder, emotion_label)
        # Parcours des fichiers d'images dans chaque dossier d'émotion
        image_count = 0
        for image_file in os.listdir(emotion_folder):
            if image_count >= max_images_per_class:
                break
            image_path = os.path.join(emotion_folder, image_file)
            # Charger l'image en niveaux de gris
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            # Ajouter l'image et l'étiquette aux listes correspondantes
            images_list.append(image)
            labels_list.append(emotion_label)
            image_count += 1

    # Encodage des étiquettes d'émotions en valeurs numériques
    label_encoder = LabelEncoder()
    labels_list = label_encoder.fit_transform(labels_list)

    # Convertir les images en tableaux numpy
    images_list = np.array(images_list)

    # Mélange aléatoire des données
    images_list, labels_list = shuffle(images_list, labels_list, random_state=42)

    if(type=="train"):
        # Utilisation de SMOTE pour sur-échantillonner les classes minoritaires
        smote = SMOTE(random_state=42)
        images_list, labels_list = smote.fit_resample(images_list.reshape(len(images_list), -1), labels_list)

    # Affichage des statistiques
    print("Nombre total d'images :", len(images_list))
    print()

    # Appeler la fonction pour compter les échantillons dans chaque classe
    count_samples(labels_list)

In [17]:
# Chemin vers le dossier contenant les images de test
train_data_folder = "test"  # Mettez le bon chemin vers votre dossier de test
test_data_folder = "D:/projet python/FAA_project/data"  # Mettre le bon chemin
emotion_labels = ["anger", "disgust", "fear", "happiness", "neutral", "sadness", "surprise"]

# Charger les données d'entraînement
train_images = []
train_labels = []
process_data(train_data_folder, emotion_labels, train_images, train_labels, "train")

# Charger les données de test
test_images = []
test_labels = []
process_data(test_data_folder, emotion_labels, test_images, test_labels, "test")

Nombre total d'images : 12936

Classe 0: 1848 échantillons
Classe 1: 1848 échantillons
Classe 2: 1848 échantillons
Classe 3: 1848 échantillons
Classe 4: 1848 échantillons
Classe 5: 1848 échantillons
Classe 6: 1848 échantillons
Nombre total d'images : 2221

Classe 0: 130 échantillons
Classe 1: 220 échantillons
Classe 2: 131 échantillons
Classe 3: 360 échantillons
Classe 4: 290 échantillons
Classe 5: 220 échantillons
Classe 6: 870 échantillons


In [18]:

# Appliquer PCA sur les données d'entraînement
pca = PCA(n_components=100)  # Définir le nombre de composantes principales souhaité

# Convertir la liste d'images en un tableau numpy
train_images_array = np.array(train_images)

# Aplatir les images d'entraînement
train_images_flat = train_images_array.reshape(len(train_images_array), -1)

# Appliquer PCA sur les images aplaties
train_images_pca = pca.fit_transform(train_images_flat)

In [19]:

# Créer un modèle SVM
svm_model = SVC(probability=True)

# Entraîner le modèle sur les données d'entraînement PCA
svm_model.fit(train_images_pca, train_labels)

# Sauvegarder le modèle entraîné
joblib.dump(svm_model, 'svm_model.pkl')

# Charger le modèle à partir du fichier
loaded_svm_model = joblib.load('svm_model.pkl')

In [20]:

# Convertir la liste d'images de test en un tableau numpy
test_images_array = np.array(test_images)

# Aplatir les images de test
test_images_flat = test_images_array.reshape(len(test_images_array), -1)

# Appliquer PCA sur les images aplaties de test
test_images_pca = pca.transform(test_images_flat)

In [21]:
from sklearn.metrics import classification_report

# Calculer les prédictions pour les données de test
predictions = loaded_svm_model.predict(test_images_pca)

# Obtenir le rapport de classification
report = classification_report(test_labels, predictions, target_names=emotion_labels)

# Afficher le rapport de classification
print(report)

              precision    recall  f1-score   support

       anger       0.16      0.36      0.23       130
     disgust       0.43      0.55      0.48       220
        fear       0.00      0.00      0.00       131
   happiness       0.44      0.43      0.43       360
     neutral       0.27      0.36      0.31       290
     sadness       0.11      0.10      0.10       220
    surprise       0.87      0.52      0.65       870

    accuracy                           0.41      2221
   macro avg       0.33      0.33      0.32      2221
weighted avg       0.51      0.41      0.44      2221

